# MLP

## Gradient of the cost function

### Unvectorized

We work through calculating the gradient of the cost function for a 2 layer MLP for one example.

$\textbf{x} \in \mathbb{R}^{p^{[1]}}$

$\textbf{W}^{[1]} \in \mathbb{R}^{p^{[2]} \times p^{[1]}}$

$\textbf{W}^{[2]} \in \mathbb{R}^{1 \times p^{[2]}}$ (the weights in the last layer form a row vector)

$y \in \{0, 1\}$

$\textbf{a}^{[0]} = \textbf{x}$

$\textbf{z}^{[1]} = \textbf{W}^{[1]} \textbf{a}^{[0]} \in \mathbb{R}^{p^{[2]}}$

$\textbf{a}^{[1]} = g^{[1]}(\textbf{z}^{[1]}) \in \mathbb{R}^{p^{[2]}}$

$z^{[2]} = \textbf{W}^{[2]} \textbf{a}^{[1]} \in \mathbb{R}$

$a^{[2]} = g^{[2]}(z^{[2]}) \in \mathbb{R}$

$l = -y \log a^{[2]} - (1-y) \log (1 - a^{[2]})$

$\frac{\partial l}{\partial W_j^{[2]}} = \frac{\partial l}{\partial a^{[2]}} \frac{\partial a^{[2]}}{\partial z^{[2]}} \frac{\partial z^{[2]}}{\partial W_j^{[2]}}$

$\frac{\partial l}{\partial a^{[2]}} = -\frac{y}{a^{[2]}} + \frac{1-y}{1-a^{[2]}}$

$\frac{\partial a^{[2]}}{\partial z^{[1]}} = a^{[2]} (1 - a^{[2]})$

$\frac{z^{[2]}}{\partial W_j^{[2]}} = \frac{\partial}{\partial W_j^{[2]}} \left(W_1^{[2]} a_1^{[1]} + \dots + W_{p^{[2]}}^{[2]} a_{p^{[2]}}^{[1]} \right) = a_j^{[1]}$

$\frac{\partial l}{\partial z^{[2]}} = a^{[2]} - y$

$\frac{\partial l}{\partial W_j^{[2]}} = \frac{\partial l}{\partial z^{[2]}} a_j^{[1]}$

$\frac{\partial l}{\partial W_{i,j}^{[1]}} = \frac{\partial l}{\partial a^{[2]}} \frac{\partial a^{[2]}}{\partial z^{[2]}} \frac{\partial z^{[2]}}{\partial a^{[1]}} \frac{\partial a^{[1]}}{\partial z^{[1]}} \frac{\partial z^{[1]}}{\partial W_{i,j}^{[1]}}$

$\frac{\partial l}{\partial z^{[1]}} = \frac{\partial l}{\partial z^{[2]}} W^{[2]} g'^{[1]}(z^{[1]})$

$\frac{\partial l}{\partial W_{i,j}^{[1]}} = \frac{\partial l}{\partial z^{[1]}} \frac{\partial z^{[1]}}{\partial W_{i,j}^{[1]}}$

### Vectorized (p_out, p_in)

We start with $\frac{\partial J}{\partial \textbf{Z}^{[L]}} = \textbf{A}^{[L]} - \textbf{y}$. Then proceeding backwards through the layers, we compute $\frac{\partial J}{\partial \textbf{W}^{[l]}} = \frac{1}{n^{[l-1]}} \frac{\partial J}{\partial \textbf{Z}^{[l]}} (\textbf{A}^{[l-1]})^T$ and $\frac{\partial J}{\partial \textbf{Z}^{[l]}} = (\textbf{W}^{[l+1]})^T (\frac{\partial J}{\partial \textbf{Z}^{[l+1]}}) \odot g'^{[l]}(\textbf{Z}^{[l]})$.

## Layer implementation

In [7]:
import numpy as np

In [8]:
class Sigmoid:
    
    def __init__(self):
        self._cache = {}
        
    def forward(self, x):
        """Forward pass for the sigmoid function.
        
        Args:
        - x: (*), where * means any number of dimensions.
        
        Returns:
        - y: (*), same shape as the input.
        
        Sources:
        * https://github.com/scipy/scipy/blob/main/scipy/special/_logit.h#L14
        * https://discuss.pytorch.org/t/where-is-the-real-implementation-of-codes-of-sigmoid/156417
        * https://github.com/pytorch/pytorch/blob/main/aten/src/ATen/native/cuda/UnarySpecialOpsKernel.cu#L123-L158
        * https://github.com/pytorch/pytorch/blob/main/aten/src/ATen/native/cpu/UnaryOpsKernel.cpp#L37-L68
        """
        self._cache['y'] = 1./(1. + np.exp(-x))
        return self._cache['y']

    def backward(self, dy):
        """Backward pass for the sigmoid function.

        Args:
        - dy: (*), where * means any number of dimensions.

        Returns:
        - dx: (*), same shape as the input.
        """
        return dy * self._cache['y'] * (1. - self._cache['y'])

In [9]:
class ReLU:
    
    def __init__(self):
        self._cache = {}
        
    def forward(self, x):
        """Forward pass for the ReLU function.
        
        Args:
        - x: (*), where * means any number of dimensions.
        
        Returns:
        - y: (*), same shape as the input.
        """
        z = np.maximum(x, 0)
        self._cache['z'] = z
        return z
    
    def backward(self, dy):
        """Backward pass for the ReLU function.
        
        Args:
        - dy: (*), where * means any number of dimensions.
        
        Returns:
        - dx: (*), same shape as the input.
        """
        m = self._cache['z'] > 0
        dx = dy * m
        return dx

In [10]:
class Linear:

    def __init__(self, p_in, p_out, rng):
        self.weight = rng.normal(size=(p_in, p_out)) * (1/np.sqrt(p_in))
        self._cache = {}

    def forward(self, x):
        """Forward pass for linear layer.

        Args:
        - x: (n, p_in)

        Returns:
        - y: (n, p_out)
        """
        self._cache['x'] = x
        # [n, p_out] = [n, p_in] [p_in, p_out]
        return x @ self.weight

    def backward(self, dy):
        """Backward pass for linear layer.

        Args:
        - dy: (n, p_out)

        Returns:
        - dw: (p_in, p_out)
        - dx: (n, p_in)
        """
        # [p_in, p_out] = [p_in, n] [n, p_out]
        dw = (1./self._cache['x'].shape[0]) * self._cache['x'].T @ dy
        # [n, p_in] = [n, p_out] [p_out, p_in] 
        dx = dy @ self.weight.T
        return dw, dx

In [11]:
class BCEWithLogitsLoss:
    """Binary cross-entropy from logits.
    
    Sources:
    * https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html
    """
    
    def __init__(self):
        self._sigmoid = Sigmoid()
        self._cache = {}
        
    def forward(self, z, y):
        """Forward pass for binary cross-entropy from logits.
        
        The binary cross-entropy loss can be computed as follows:
        
        ```
        loss = -1 * y * np.log(g(z)) - (1 - y) * np.log(1 - g(z))
        ```
        
        where g(z) is the sigmoid function.
        
        We can write the sigmoid function as:
        
        ```
        np.exp(z) / (np.exp(0) + np.exp(z))
        ```
        
        In this way, we can see that the sigmoid function is equivalent
        to the softmax over 0 and z, so we can use the log-sum-exp trick:
        
        ```
        z1 = np.zeros((len(z), 2))
        z1[:, 0] = z[:, 0]
        y1 = np.zeros((len(y), 2))
        y1[:, 0] = y[:, 0]
        y1[:, 1] = 1 - y[:, 0]
        logprobs = z1 - np.log(np.sum(np.exp(z1), axis=1, keepdims=True))
        loss = -1 * np.sum(y1 * logprobs, axis=1, keepdims=True)
        ```
        
        Here are the columns of logprobs:
        
        ```
        1st column: z - np.log(np.exp(z) + 1)
        2nd column: -1 * np.log(np.exp(z) + 1)
        ```
        
        We can write the loss more succinctly as:
        
        ```
        loss = -1 * y * (z - np.log(np.exp(z) + 1)) - (1 - y) * (-1 * np.log(np.exp(z) + 1))
        ```
        
        Simplifying further:
        
        ```
        -yz + y log(exp(z) + 1) + (1 - y) log(exp(z) + 1)
        -yz + y log(exp(z) + 1) + log(exp(z) + 1) - y log(exp(z) + 1) 	
        loss = -1 * y * z + np.log(np.exp(z) + 1)
        ```
        
        This also works:
        
        ```
        loss = (1 - y) * z + np.log(np.exp(-z) + 1)
        ```
        
        Handling negative and positive cases separately:
        
        ```
        z < 0: loss = -1 * y * z + np.log(np.exp(z) + 1)
        otherwise: loss = (1 - y) * z + np.log(np.exp(-z) + 1)
        ```
        
        Avoid taking exponentials of positive values to avoid overflow:
        
        ```
        t = -np.maximum(z, 0)
        loss = (1 - y) * z + t + np.log(np.exp(-t) + np.exp(-z - t))
        ```
        
        Args:
        - z: (n, 1). The logits.
        - y: (n, 1). The binary targets.
        
        Returns:
        - mean_loss: ()
        
        Sources:
        * https://stackoverflow.com/questions/66906884/how-is-pytorchs-class-bcewithlogitsloss-exactly-implemented
        * https://github.com/pytorch/pytorch/blob/main/aten/src/ATen/native/Loss.cpp#L356
        """
        self._cache['z'] = z
        self._cache['y'] = y
        t = -np.maximum(z, 0)
        loss = (1 - y) * z + t + np.log(np.exp(-t) + np.exp(-z - t))
        return np.mean(loss)
    
    def backward(self):
        """Backward pass for binary cross-entropy from logits.
        
        a = g(z)
        L = -y * log(a) - (1 - y) * log(1 - a)
        
        dLdz = dLda dadz
        dLda = -y/a + (1 - y)/(1 - a)
        dadz = a (1 - a)
        dLdz = a - y 
        """
        a = self._sigmoid.forward(self._cache['z'])
        return (a - self._cache['y'])


In [17]:
import pytest
import torch


# http://ufldl.stanford.edu/tutorial/supervised/DebuggingGradientChecking/
_EPS = 1e-4


rng = np.random.default_rng(seed=0)
n = 10
p_in = 4
p_out1 = 3
p_out2 = 1
x = rng.normal(0, 1, (n, p_in))

l1 = Linear(p_in, p_out1, rng)
l2 = Linear(p_out1, p_out2, rng)
g1 = ReLU()
z = l2.forward(g1.forward(l1.forward(x)))

p = Sigmoid().forward(z)
y = rng.binomial(1, p)

loss = BCEWithLogitsLoss()
actual_loss = loss.forward(z, y)

dy = loss.backward()
actual_dw2, dx = l2.backward(dy)
dx = g1.backward(dx)
actual_dw1, _ = l1.backward(dx)

# torch

l1_ = torch.nn.Linear(p_in, p_out1, bias=False)
l2_ = torch.nn.Linear(p_out1, p_out2, bias=False)
with torch.no_grad():
    l1_.weight[:] = torch.tensor(l1.weight.T).float()
    l2_.weight[:] = torch.tensor(l2.weight.T).float()
x_ = torch.tensor(x).float()
z_ = l2_(torch.relu(l1_(x_)))
y_ = torch.tensor(y).float()
loss_ = torch.nn.BCEWithLogitsLoss()
out = loss_(z_, y_)
out.backward()
expected_loss = out.tolist()
expected_dw1 = np.array(l1_.weight.grad.T.tolist())
expected_dw2 = np.array(l2_.weight.grad.T.tolist())

np.testing.assert_almost_equal(actual_loss, expected_loss)
np.testing.assert_almost_equal(actual_dw1, expected_dw1)
np.testing.assert_almost_equal(actual_dw2, expected_dw2)

# Gradient checking

for w, actual_dw in [
    (l1.weight, actual_dw1),
    (l2.weight, actual_dw2)
]:
    approx_dw = np.zeros(w.shape)
    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            # Why 2-sided gradient checking?
            # https://stats.stackexchange.com/questions/318380/why-is-two-sided-gradient-checking-more-accurate

            w[i,j] += _EPS
            z_plus = l2.forward(g1.forward(l1.forward(x)))
            l_plus = loss.forward(z_plus, y)
            w[i,j] -= _EPS

            w[i,j] -= _EPS
            z_minus = l2.forward(g1.forward(l1.forward(x)))
            l_minus = loss.forward(z_minus, y)
            w[i,j] += _EPS

            approx_dw[i,j] = (l_plus - l_minus) / (2 * _EPS)

    np.testing.assert_almost_equal(actual_dw, approx_dw)
    
print("OK")

OK


## Function Implementation

### (p_out, p_in)

* hidden units are stacked vertically and examples are stacked horizontally
* weight matrices therefore have the form (p_out, p_in)

In [2]:
import tensorflow as tf
import numpy as np

2022-09-07 09:08:06.608787: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def relu(x):
    return np.maximum(x, 0)

def sigmoid(x):
    return 1. / (1 + np.exp(-x))

In [4]:
def forward(X, weights):
    cache = []
    A = X.T
    for l in range(len(weights)):
        W = weights[l].numpy().T
        Z = np.dot(W, A)
        cache.append((A, W, Z))
        if l == (len(weights) - 1):
            A = sigmoid(Z)
        else:
            A = relu(Z)
    return A, cache

In [5]:
def backward(AL, y, cache):
    grads = []
    num_layers = len(cache)
    for l in range(num_layers - 1, -1, -1):
        A_prev, _, Z = cache[l]

        if l == num_layers - 1:
            dZ = (AL - y.T)
        else:
            _, W, _ = cache[l+1]
            dA = np.dot(W.T, dZ)
            dAdZ = np.zeros(Z.shape)
            dAdZ[Z > 0] = 1
            dZ = dA * dAdZ
        dW = (1./A_prev.shape[1]) * np.dot(dZ, A_prev.T)
        grads.append(dW)
    return grads[::-1]

In [9]:
class Model(tf.keras.Model):

    def __init__(self):
        super(Model, self).__init__()
        self.d1 = tf.keras.layers.Dense(16, activation='relu', use_bias=False)
        self.d2 = tf.keras.layers.Dense(8, activation='relu', use_bias=False)
        self.d3 = tf.keras.layers.Dense(1, activation='sigmoid', use_bias=False)

    def call(self, x):
        x = self.d1(x)
        x = self.d2(x)
        return self.d3(x)

def assert_close(actual, expected):
    TOL = 1e-3
    assert (abs(actual - expected) < TOL).all()

def test_mlp():
    np.random.seed(0)
    tf.random.set_seed(0)

    n = 10
    p = 32

    X = np.random.random((n, p))
    p = np.random.random((n,))
    y = np.zeros((n, 1))
    for i in range(n):
        r = np.random.random()
        if r <= p[i]:
            y[i, 0] = 1.

    model = Model()
    loss_object = tf.keras.losses.BinaryCrossentropy()
    with tf.GradientTape() as tape:
        out = model(X)
        loss = loss_object(y, out)
    expected_grads = tape.gradient(loss, model.trainable_variables)

    expected_AL = model(X).numpy()
    actual_AL, cache = forward(X, model.weights)
    actual_grads = backward(actual_AL, y, cache)

    assert_close(actual_AL.T, expected_AL)
    assert len(actual_grads) == len(expected_grads)
    for i in range(len(expected_grads)):
        assert_close(actual_grads[i].T, expected_grads[i].numpy())

test_mlp()

### (p_in, p_out)

In [6]:
import numpy as np
import tensorflow as tf

In [7]:
def relu(x):
    return np.maximum(x, 0)

def sigmoid(x):
    return 1. / (1 + np.exp(-x))

In [8]:
def forward(X, weights):
    cache = []
    A = X
    for l in range(len(weights)):
        W = weights[l].numpy()
        Z = np.dot(A, W)
        cache.append((A, W, Z))
        if l == (len(weights) - 1):
            A = sigmoid(Z)
        else:
            A = relu(Z)
    return A, cache

In [9]:
def backward(AL, y, cache):
    grads = []
    num_layers = len(cache)
    for l in range(num_layers - 1, -1, -1):
        A_prev, _, Z = cache[l]

        if l == num_layers - 1:
            dZ = (AL - y)
        else:
            _, W, _ = cache[l+1]
            dA = np.dot(dZ, W.T)
            dAdZ = np.zeros(Z.shape)
            dAdZ[Z > 0] = 1
            dZ = dA * dAdZ
        # dW is simpler than the equations for dZ
        dW = (1./A_prev.shape[0]) * np.dot(A_prev.T, dZ)
        grads.append(dW)
    return grads[::-1]

In [10]:
class Model(tf.keras.Model):

    def __init__(self):
        super(Model, self).__init__()
        self.d1 = tf.keras.layers.Dense(16, activation='relu', use_bias=False)	
        self.d2 = tf.keras.layers.Dense(8, activation='relu', use_bias=False)
        self.d3 = tf.keras.layers.Dense(1, activation='sigmoid', use_bias=False)

    def call(self, x):
        x = self.d1(x)
        x = self.d2(x)
        return self.d3(x)

def assert_close(actual, expected):
    TOL = 1e-3
    assert (abs(actual - expected) < TOL).all()

def test_mlp():
    np.random.seed(0)
    tf.random.set_seed(0)

    n = 10
    p = 32

    X = np.random.random((n, p))
    p = np.random.random((n,))
    y = np.zeros((n, 1))
    for i in range(n):
        r = np.random.random()
        if r <= p[i]:
            y[i, 0] = 1.

    model = Model()
    loss_object = tf.keras.losses.BinaryCrossentropy()
    with tf.GradientTape() as tape:
        out = model(X)
        loss = loss_object(y, out)
    expected_grads = tape.gradient(loss, model.trainable_variables)

    expected_AL = model(X).numpy()
    actual_AL, cache = forward(X, model.weights)
    actual_grads = backward(actual_AL, y, cache)

    assert_close(actual_AL, expected_AL)
    assert len(actual_grads) == len(expected_grads)
    for i in range(len(expected_grads)):
        assert_close(actual_grads[i], expected_grads[i].numpy())

test_mlp()

## Sources

* [Gradient Descent For Neural Networks (C1W3L09)](https://www.youtube.com/watch?v=7bLEWDZng_M&list=PLkDaE6sCZn6Ec-XTbcX1uRg2_u4xOEky0&index=33)
* [Backpropagation Intuition (C1W3L10)](https://www.youtube.com/watch?v=yXcQ4B-YSjQ&list=PLkDaE6sCZn6Ec-XTbcX1uRg2_u4xOEky0&index=35)
* https://towardsdatascience.com/lets-code-a-neural-network-in-plain-numpy-ae7e74410795